In [304]:
import pandas as pd
import numpy as np

In [2]:
amazon = pd.read_csv("amazon.csv")

In [435]:
amazon[amazon['Author'] == "Laura Hillenbrand"]

,Name,Author,User Rating,Reviews,Price,Price_r,Year,Genre
608,"Unbroken: A World War II Story of Survival, Re...",Laura Hillenbrand,4.8,29673,$16.00,16,2010,Non Fiction
609,"Unbroken: A World War II Story of Survival, Re...",Laura Hillenbrand,4.8,29673,$16.00,16,2011,Non Fiction
610,"Unbroken: A World War II Story of Survival, Re...",Laura Hillenbrand,4.8,29673,$16.00,16,2012,Non Fiction
611,"Unbroken: A World War II Story of Survival, Re...",Laura Hillenbrand,4.8,29673,$13.00,13,2014,Non Fiction
612,"Unbroken: A World War II Story of Survival, Re...",Laura Hillenbrand,4.8,29673,$16.00,16,2014,Non Fiction


## Preguntas

Por año cuantos libros ha publicado un autor, recuerde que los años no trabajados deberia de aparecer en cero. considere como primer registro por autor el primer año en que publico y el ultimo el año maximo encontrado.

Ejemplo, si Uayeb public en 2009 dos libros, y en 2011 tres. y esas son las dos unicas entradas para este autor. la salida para el deberia de ser:

In [30]:
pd.DataFrame({ "Autor" : ["Uayeb","Uayeb","Uayeb"] , "Year" : [2009,2010,2011]  , "Conteo" : [2,0,3] })

,Autor,Year,Conteo
0,Uayeb,2009,2
1,Uayeb,2010,0
2,Uayeb,2011,3


In [82]:
amazon['Year'].drop_duplicates()

0     2016
1     2011
2     2018
3     2021
4     2017
5     2019
7     2014
12    2010
13    2020
16    2009
18    2015
24    2013
37    2012
Name: Year, dtype: int64

In [234]:
sol_ = amazon.groupby( ["Author", "Year"], as_index = False ).agg({ 'Name' : ['count'] })
sol_.columns = sol_.columns.droplevel(1)
sol_.columns = ["Author", "Year", "Conteo"]

In [235]:
tmp_ = sol_.groupby( by = "Author", as_index = False).agg( {'Year' : ['min', 'max'] })
tmp_.columns = tmp_.columns.droplevel(1)
tmp_.columns = ["Author", "Min Year", "Max Year"]

In [236]:
for author_ in sol_.loc[ :, 'Author'].drop_duplicates():
    for anio_ in amazon.loc[ :, 'Year' ].drop_duplicates():
        datatmp = tmp_.loc[tmp_['Author'] == author_, :] ## Cargamos temporalmente la data del autor
        for anioMax in datatmp.loc[:, 'Max Year']: ## Obtenemos el año máximo de ese autor
            continue
        for anioMin in datatmp.loc[:, 'Min Year']: ## Obtenemos el año minimo de ese autor
            continue
        if ((anio_ >= anioMin) & (anio_ <= anioMax)):
            dataTMP = sol_.loc[ (sol_['Author'] == author_) & (sol_['Year'] == anio_), : ]
            if dataTMP.empty:
                aniosNoJob = pd.DataFrame({
                    'Author' : [author_],
                    'Year' : [anio_],
                    'Conteo' : [0]
                })
                sol_ = pd.concat( [sol_,aniosNoJob] )
        

In [237]:
sol_ = sol_.sort_values(["Author","Year"], ascending = True) 
sol_.columns = ["Autor","Year","Conteo"] 

In [238]:
amazon[ amazon['Author'] == "Stieg Larsson" ] ## Para mostrar la información de Stieg Larsson, con la data sin manipular

,Name,Author,User Rating,Reviews,Price,Price_r,Year,Genre
459,The Girl Who Kicked the Hornet's Nest (Millenn...,Stieg Larsson,4.7,7747,$14.00,14,2010,Fiction
460,The Girl Who Kicked the Hornet's Nest (Millenn...,Stieg Larsson,4.7,7747,$14.00,14,2011,Fiction
461,The Girl Who Played with Fire (Millennium Series),Stieg Larsson,4.7,7251,$9.00,9,2010,Fiction
462,The Girl Who Played with Fire (Millennium),Stieg Larsson,4.7,7251,$16.00,16,2009,Fiction
463,The Girl with the Dragon Tattoo (Millennium Se...,Stieg Larsson,4.4,10559,$2.00,2,2009,Fiction
464,The Girl with the Dragon Tattoo (Millennium Se...,Stieg Larsson,4.4,10559,$2.00,2,2010,Fiction


In [239]:
sol_[ sol_['Autor'] == "Abraham Verghese" ] ## Para mostrar la información de Stieg Larsson, con la data manipulada 

,Autor,Year,Conteo
0,Abraham Verghese,2010,1
1,Abraham Verghese,2011,1


In [241]:
sol_.shape

(712, 3)

Que Autores han trabado en un año en mas de un libro

Ejemplo de salida esperada:

In [23]:
pd.DataFrame({
    
    "Autor" : ['Uayeb','Imix']
    , "Year" : [2010, 2010]
    , "Conteo de Titulos" : [2 , 3]
    
})

,Autor,Year,Conteo de Titulos
0,Uayeb,2010,2
1,Imix,2010,3


In [89]:
sol2_ = sol_.loc[ sol_['Conteo'] > 1, : ]

In [95]:
sol2_.columns = ["Autor", "Year", "Conteo de Titulos"]

In [436]:
sol2_.head(5)

,Autor,Year,Conteo de Titulos
5,Adam Wallace,2020,2
16,Amanda Gorman,2021,2
58,Bill Martin Jr.,2020,2
61,Bill O'Reilly,2012,2
67,Blue Star Coloring,2015,2


Que genero por año ha obtenido el mejor rating en promedio. (solo deberia de obtener un registro por año)

- ejemplo de la salida esperada:

In [22]:
pd.DataFrame({
    "Year" : [2009,2010]
    , "Genre" : [ "Fiction" , "Fiction" ]
    , "Mean User Rating": [ 4.59 , 4.61 ]
})

,Year,Genre,Mean User Rating
0,2009,Fiction,4.59
1,2010,Fiction,4.61


In [440]:
amazon['Genre'].drop_duplicates()

0    Non Fiction
1        Fiction
Name: Genre, dtype: object

In [441]:
sol3_ = amazon.groupby( ["Genre", "Year"], as_index = False ).agg( { 'User Rating' : ['mean'] } )
sol3_.columns = sol3_.columns.droplevel(1)
sol3_.columns = ["Genre", "Year", "Mean User Rating"]

In [442]:
sol3_['Rank'] = sol3_.sort_values(["Year", "Mean User Rating"], ascending = [True, False]).groupby(["Year"]).cumcount()

In [443]:
sol3_ = sol3_.loc[ sol3_['Rank'] == 0, ["Year", "Genre", "Mean User Rating"] ]

In [444]:
sol3_.shape

(13, 3)

In [445]:
sol3_.head(5)

,Year,Genre,Mean User Rating
0,2009,Fiction,4.591667
1,2010,Fiction,4.615000
2,2011,Fiction,4.619048
5,2014,Fiction,4.631034
6,2015,Fiction,4.652941


Cuales son los autores que por lo menos dos años consecutivos hayan tenido en promedio ratings que decrecen 

Ejemplo

- Si Uayeb caballero, en 2010 saco dos libros, y el rating de esos dos libros es de 3.1, ese mismo autor en 2011 en su unico libro publicado ese año su promedio fue de 2.9 y en 2012 sus tres libros publicados en promedio su rating es de 2.8. Podemos ver que son dos años que este usuario ha ido decreciendo 

No importa que haya publicado en años salteados, ejemplo, 2010 si, 2011 no, 2012 no, 2013 si, 2014 si. para este caso el analisis comienza en 2010, 2013 y 2014

Ejemplo de salida esperada:

In [24]:
pd.DataFrame({ "autores" : ['Uayeb','Imix'] })

,autores
0,Uayeb
1,Imix


In [291]:
sol4_ = amazon.groupby(["Author", "Year"], as_index = False).agg({ 'User Rating': ['mean'] })

In [292]:
sol4_.columns = sol4_.columns.droplevel(1)

In [293]:
sol4_.columns = ["Author", "Year", "Mean Rating"]

In [294]:
sol4_.columns

Index(['Author', 'Year', 'Mean Rating'], dtype='object')

In [295]:
dfDecadencia = pd.DataFrame()

In [296]:
for author_ in amazon.loc[:, 'Author'].drop_duplicates():
    dataAuthor = sol4_.loc[sol4_['Author'] == author_, :]
    ratingTMP = 0
    contador = 0
    for mean_rating in dataAuthor.loc[:, 'Mean Rating']:
        if mean_rating < ratingTMP:
            contador = contador + 1
        ratingTMP = mean_rating
    if contador >= 2:
        dfDecadencia = pd.concat([dfDecadencia, dataAuthor['Author']])

In [298]:
dfDecadencia.columns = ["Author"]

In [300]:
dfDecadencia = dfDecadencia.drop_duplicates()

In [301]:
dfDecadencia

,Author
85,Charlaine Harris
60,Bill O'Reilly


Cuales son los libros que por año se encuentran arriba del percentil 70. (Sin importar genero)

Ejemplo de salida esperada

In [26]:
pd.DataFrame({
    "Year": [2010,2010,2011,2011]
    , "Titulos": ["Libro A","Libro B","Libro Z","Libro Y"]
    , "Autor": ["Autor 1","Autor 2","Autor 1","Autor 3"]
    , "Rating": [4.1, 4.0,4.2,4.3]
    , "Rating .70": [3.8,3.8,3.7,3.7]
})

,Year,Titulos,Autor,Rating,Rating .70
0,2010,Libro A,Autor 1,4.1,3.8
1,2010,Libro B,Autor 2,4.0,3.8
2,2011,Libro Z,Autor 1,4.2,3.7
3,2011,Libro Y,Autor 3,4.3,3.7


Este ejemplo es para que se haga una idea de la estructura de salida esperada, claramente por año dependiendo de las publicaciones pueden haber varios libros arriba del percentil 70


In [422]:
sol5_ = amazon.loc[:, ["Author", "Name", "Year", "User Rating"]]

In [423]:
sol5_ = sol5_.sort_values(["Year", "Author"], ascending = [True, True])

In [424]:
Percentiles = sol5_.groupby('Year', as_index = False)['User Rating'].quantile(0.7)

In [425]:
Percentiles

,Year,User Rating
0,2009,4.70
1,2010,4.70
2,2011,4.70
3,2012,4.70
4,2013,4.73
5,2014,4.80
6,2015,4.80
7,2016,4.80
8,2017,4.80
9,2018,4.80


In [427]:
## Haciendo uso de numpy para verificar lo obtenido para 2021
datatmp =  sol5_[sol5_['Year'] == 2021] ##
perPrueba = np.percentile(datatmp['User Rating'], 70)
perPrueba

4.8

In [428]:
sol5_ = sol5_.merge(Percentiles, on = "Year", how = "inner")

In [429]:
sol5_.columns = ["Autor", "Titulos", "Year", "Rating", "Rating .70"]

In [430]:
result = sol5_.loc[ sol5_['Rating'] > sol5_['Rating .70'], ["Year", "Titulos", "Autor", "Rating", "Rating .70"] ]

In [431]:
result[result['Year'] == 2009].shape

(11, 5)

In [432]:
sol5_[sol5_['Year'] == 2009].shape

(50, 5)

In [439]:
result.shape

(109, 5)

In [438]:
result.head(5)

,Year,Titulos,Autor,Rating,Rating .70
0,2009,Watchmen,Alan Moore,4.8,4.7
18,2009,Diary of a Wimpy Kid: The Last Straw (Book 3),Jeff Kinney,4.8,4.7
19,2009,"Dog Days (Diary of a Wimpy Kid, Book 4) (Volum...",Jeff Kinney,4.8,4.7
21,2009,"Mastering the Art of French Cooking, Vol. 2",Julia Child,4.8,4.7
22,2009,The Help,Kathryn Stockett,4.8,4.7
